In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from transformer import *
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Pickle

In [26]:
import pickle

try:
    table = pd.read_pickle('../association_table.pkl')
except:
    table = pd.read_excel('../Cas_D_Etude/Cas_Etude_Volcano/NE PAS OUVRIR/association_table.xlsx')
    table.to_pickle("../Cas_D_Etude/Cas_Etude_Vulcano/association_table.pkl")

# balanced_subsampler

In [27]:
from transformer import Event
from subsampler import balanced_subsampler

table = Event().fit_transform(table)
balanced_table = balanced_subsampler(table, 100)
balanced_table.event.value_counts()

TOR    100
TR     100
PIS    100
HIB    100
VT     100
EXP    100
LP     100
Name: event, dtype: int64

In [33]:
balanced_table.to_pickle("/home/d5learner-08/GDrive/Cas_D_Etude/Cas_Etude_Vulcano/association_balanced_table.pkl")

In [40]:
balanced_table.head(5)

,Index,Event,Duration,Event start,Event stop,File name,File start,File end,Sampling rate,Overlap,Unnamed: 10,Unnamed: 11,Unnamed: 12,npts,energy,path,event
68058,68058,TR2,354,2007-04-23 03:16:13.990000+00:00,2007-04-23 03:22:07.990000+00:00,20070423_0300.UB1.SL.Z.sac,2007-04-23 03:00:00.019000+00:00,2007-04-23 03:59:59.969000+00:00,40.0,False,NaN,NaN,NaN,143999.0,0.72632,C:/Users/EC265595/Documents/Data/Ubinas_2006_2...,TR
100084,100084,PIS,27,2008-02-17 06:17:15.150000+00:00,2008-02-17 06:17:42.150000+00:00,20080217_0600.UB1.SL.Z.sac,2008-02-17 06:00:00.036000+00:00,2008-02-17 07:00:00.036000+00:00,50.0,False,NaN,NaN,NaN,180001.0,0.39290,C:/Users/EC265595/Documents/Data/Ubinas_2006_2...,PIS
80771,80771,TR1,2701,2007-06-16 11:00:10.670000+00:00,2007-06-16 11:45:11.670000+00:00,20070616_1100.UB1.SL.Z.sac,2007-06-16 10:59:59.993000+00:00,2007-06-16 11:59:59.993000+00:00,100.0,False,NaN,NaN,NaN,360001.0,11.18200,C:/Users/EC265595/Documents/Data/Ubinas_2006_2...,TR
121808,121808,TOR,27,2011-01-22 05:40:34.710000+00:00,2011-01-22 05:41:01.710000+00:00,20110122_0500.UB1.SL.Z.sac,2011-01-22 05:00:00.043000+00:00,2011-01-22 06:00:00.043000+00:00,100.0,False,NaN,NaN,NaN,360001.0,0.02427,C:/Users/EC265595/Documents/Data/Ubinas_2006_2...,TOR
115494,115494,VT1,27,2009-12-02 02:50:06.150000+00:00,2009-12-02 02:50:33.150000+00:00,20091202_0200.UB1.SL.Z.sac,2009-12-02 02:00:00.036000+00:00,2009-12-02 03:00:00.036000+00:00,50.0,False,NaN,NaN,NaN,180001.0,69.37142,C:/Users/EC265595/Documents/Data/Ubinas_2006_2...,VT


In [44]:
path = "data/Extracted/"

transfo = Transformer(path)
table_transf=transfo.fit_transform(balanced_table)

In [30]:
table_transf.to_pickle("/home/d5learner-08/GDrive/Cas_D_Etude/Cas_Etude_Vulcano/association_table_transfo.pkl")

In [45]:
table_transf.head(5)

,Index,Event,Duration,Event start,Event stop,Sampling rate,Overlap,npts,energy,path,event,variance,mean,median,maximum,amplitude,kurtosis,skew
68058,68058,TR2,354,2007-04-23 03:16:13.990000+00:00,2007-04-23 03:22:07.990000+00:00,40.0,False,143999.0,0.72632,data/Extracted/TR/TR_68058.npy,TR,49533.359375,-228.184677,-227.000000,945.000000,2543.000000,0.0,0.0
100084,100084,PIS,27,2008-02-17 06:17:15.150000+00:00,2008-02-17 06:17:42.150000+00:00,50.0,False,180001.0,0.39290,data/Extracted/PIS/PIS_100084.npy,PIS,14347.152344,53.722961,53.000000,538.000000,976.000000,0.0,0.0
80771,80771,TR1,2701,2007-06-16 11:00:10.670000+00:00,2007-06-16 11:45:11.670000+00:00,100.0,False,360001.0,11.18200,data/Extracted/TR/TR_80771.npy,TR,39968.671875,67.558739,68.000000,1715.000000,3429.000000,0.0,0.0
121808,121808,TOR,27,2011-01-22 05:40:34.710000+00:00,2011-01-22 05:41:01.710000+00:00,100.0,False,360001.0,0.02427,data/Extracted/TOR/TOR_121808.npy,TOR,1773.701904,0.112566,0.044842,275.282593,486.641968,0.0,0.0
115494,115494,VT1,27,2009-12-02 02:50:06.150000+00:00,2009-12-02 02:50:33.150000+00:00,50.0,False,180001.0,69.37142,data/Extracted/VT/VT_115494.npy,VT,27.193705,59.099998,59.000000,74.000000,32.000000,0.0,0.0


In [54]:
# gridsearch for the Random Forest classifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

liste_X =["Duration","energy", "mean", "median", "maximum",  "amplitude", "variance"]

y=table_transf.event.copy()
X=table_transf[liste_X]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

k_range = [60,80,100,150]
param_grid =  {'n_estimators': k_range}

RF_clf=RandomForestClassifier()

grid_search = GridSearchCV(RF_clf, param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(grid_search.best_score_)
print(grid_search.best_params_)

RF_clf.fit(X_train, y_train)


0.6723809523809523
{'n_estimators': 60}


RandomForestClassifier()

In [55]:
RF_clf.score(X_test,y_test)

0.6285714285714286

# Random forest - feature_importances

In [51]:
liste_X =["Duration","energy", "mean", "median", "maximum",  "amplitude", "variance"]

[list(a) for a in zip(RF_clf.feature_importances_,liste_X)]

[[0.22052487047439084, 'Duration'],
 [0.2109691475337638, 'energy'],
 [0.12703698251704218, 'mean'],
 [0.09941284950822, 'median'],
 [0.09838655369914899, 'maximum'],
 [0.11000794980358675, 'amplitude'],
 [0.13366164646384732, 'variance']]

# kurtosis, skew

In [17]:
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew
from transformer import Transformer

path = "data/Extracted/"

transfo = Transformer(path)
y=transfo.open_file(balanced_table,74458)


print( 'excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(y) ))
print( 'skewness of normal distribution (should be 0): {}'.format( skew(y) ))


excess kurtosis of normal distribution (should be 0): 5.791941681180415
skewness of normal distribution (should be 0): 0.25979697704315186


%store df

%store -r df

In [ ]:

df_histo=plt.hist(np_VT_400,bins=500)
plt.show()   
maxi=np.max(df_histo[0]) 
l=df_histo[0].tolist()
i=l.index(maxi)
print(df_histo[1][i])
p=max(plt.plot(np_VT_400))
print(p)

In [ ]:
def TF_stat():
    
    T = df.loc[0,'Duration'] # Durée échantillonnage
    fe = df.loc[0,'Sampling rate']  # Freq échantionnage
    echantillons = np.load(df.loc[0,'path'])

    tfd = fft(echantillons)
    N=len(echantillons)
    spectre = np.absolute(tfd)*2/N
    #L'échelle des fréquences est construite sachant que l'espacement fréquentiel de deux points de la TFD est l'inverse de la durée T :
    freq=np.arange(N)*1.0/T

In [ ]:
import math
import numpy as np
from matplotlib.pyplot import *
from numpy.fft import fft 
from scipy import fftpack

T = df.loc[0,'Duration'] # Durée échantillonnage
fe = df.loc[0,'Sampling rate']  # Freq échantionnage
echantillons = np.load(df.loc[0,'path'])

tfd = fft(echantillons)
N=len(echantillons)
spectre = np.absolute(tfd)*2/N
            

#L'échelle des fréquences est construite sachant que l'espacement fréquentiel de deux points de la TFD est l'inverse de la durée T :
freq=np.arange(N)*1.0/T
           
#On trace le spectre pour des fréquences allant de zéro à la moitié de la fréquence d'échantillonnage :

figure(figsize=(10,4))
plot(freq,spectre,'r')
xlabel('f')
ylabel('A')
axis([-0.1,fe/2,0,spectre.max()])
grid()


X = fftpack.fft(echantillons)   

